In [4]:
import numpy as np
import pandas as pd
import os
import itertools
import torch
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader

In [10]:
data_df = pd.read_csv("datasetsrip - synthetic-prcp-SM-flood-data-v1.csv.csv")
data_df.drop(['Unnamed: 0'], axis=1, inplace=True)
data_df

,Prcp1,SM1,Prcp2,SM2,Prcp3,SM3,flood1,flood2,flood3
0,1.539606,0.590441,2.511432,0.474052,4.585283,0.484656,0,0,0
1,2.274125,1.249220,7.461011,2.159259,17.690303,3.474562,1,1,1
2,1.000114,0.106579,1.842560,0.010000,3.395878,0.010000,0,0,0
3,1.360013,0.429366,2.085287,0.212607,3.610686,0.137334,0,0,0
4,1.158710,0.248820,1.282190,0.010000,1.592812,0.010000,0,0,0
...,...,...,...,...,...,...,...,...,...
360,1.357030,0.426691,2.020841,0.197245,3.431358,0.103488,0,0,0
361,1.154324,0.244887,3.097722,0.201381,6.761527,0.473908,0,0,1
362,3.315756,2.183443,2.927501,2.330118,3.422077,2.060617,1,1,0
363,1.779925,0.805979,2.532323,0.718383,4.327215,0.680561,0,0,0


In [11]:
def test_2layer_GCN(df, window = 3):
    df_list = [df[i:i+window] for i in range(0,df.shape[0]-window+1,1)]

    df_dict = {}
    df_dict['x'] = [df_list[i].iloc[:,0:6] for i in range(len(df_list))]
    df_dict['y'] = [df_list[i].iloc[-1,-3:] for i in range(len(df_list))]

    
    def create_node_features(df_x):
        Prcp1 = df_x['Prcp1'].values
        Prcp2 = df_x['Prcp2'].values
        Prcp3 = df_x['Prcp3'].values
        SM1 = df_x['SM1'].values
        SM2 = df_x['SM2'].values
        SM3 = df_x['SM3'].values
        # Stack Prcp and SM for each node
        node_1 = np.hstack((Prcp1, SM1))
        node_2 = np.hstack((Prcp2, SM2))
        node_3 = np.hstack((Prcp3, SM3))
        node_features = np.vstack((node_1, node_2, node_3))
        
        return node_features

    data_list = []
    for i in range(len(df_dict['x'])):
        data_list.append(Data(
            x=torch.tensor(create_node_features(df_dict['x'][i]), dtype=torch.float), 
            y=torch.tensor(df_dict['y'][i].values, dtype=torch.float), 
            edge_index=torch.tensor([[0,2], [1,1]], dtype=torch.long)
            ))
        
    class GCN(nn.Module):
        def __init__(self, in_channels, out_channels):
            super(GCN, self).__init__()
            self.conv1 = GCNConv(in_channels, 16)  
            self.conv2 = GCNConv(16, out_channels)

        def forward(self, x, edge_index):
            x = self.conv1(x, edge_index)
            x = torch.relu(x)
            x = self.conv2(x, edge_index)
            return torch.sigmoid(x)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GCN(in_channels=2*window, out_channels=1).to(device)
    criterion = nn.BCELoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    train_ratio = 0.75 
    train_size = int(train_ratio * len(data_list))
    train_data = data_list[:train_size]
    test_data = data_list[train_size:]

    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=32)

    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index).squeeze()
            loss = criterion(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.num_graphs
        
        print(f'Epoch {epoch+1}/{num_epochs}: Loss = {total_loss / len(train_loader.dataset):.4f}')

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.edge_index).squeeze()
            predicted_labels = (out > 0.5).long()
            correct += (predicted_labels == data.y).sum().item()
            total += data.y.size(0)
        
        accuracy = correct / total
        print(f'Test Accuracy: {accuracy:.4f}')

    del model
    torch.cuda.empty_cache()

test_2layer_GCN(data_df)

Epoch 1/10: Loss = 0.5853
Epoch 2/10: Loss = 0.4378
Epoch 3/10: Loss = 0.3817
Epoch 4/10: Loss = 0.3510
Epoch 5/10: Loss = 0.3135
Epoch 6/10: Loss = 0.2790
Epoch 7/10: Loss = 0.2543
Epoch 8/10: Loss = 0.2416
Epoch 9/10: Loss = 0.2143
Epoch 10/10: Loss = 0.1991
Test Accuracy: 0.9121


In [12]:
def test_2layer_GCN(df, window = 3):
    df_list = [df[i:i+window] for i in range(0,df.shape[0]-window+1,1)]

    df_dict = {}
    df_dict['x'] = [df_list[i].iloc[:,0:6] for i in range(len(df_list))]
    df_dict['y'] = [df_list[i].iloc[-1,-3:] for i in range(len(df_list))]

    
    def create_node_features(df_x):
        Prcp1 = df_x['Prcp1'].values
        Prcp2 = df_x['Prcp2'].values
        Prcp3 = df_x['Prcp3'].values
        SM1 = df_x['SM1'].values
        SM2 = df_x['SM2'].values
        SM3 = df_x['SM3'].values
        # Stack Prcp and SM for each node
        node_1 = np.hstack((Prcp1, SM1))
        node_2 = np.hstack((Prcp2, SM2))
        node_3 = np.hstack((Prcp3, SM3))
        node_features = np.vstack((node_1, node_2, node_3))
        
        return node_features

    data_list = []
    for i in range(len(df_dict['x'])):
        data_list.append(Data(
            x=torch.tensor(create_node_features(df_dict['x'][i]), dtype=torch.float), 
            y=torch.tensor(df_dict['y'][i].values, dtype=torch.float), 
            edge_index=torch.tensor([[0,2], [1,1]], dtype=torch.long)
            ))
        
    class GCN(nn.Module):
        def __init__(self, in_channels, out_channels):
            super(GCN, self).__init__()
            self.conv1 = GCNConv(in_channels, 16)  
            self.conv2 = GCNConv(16, out_channels)

        def forward(self, x, edge_index):
            x = self.conv1(x, edge_index)
            x = torch.relu(x)
            x = self.conv2(x, edge_index)
            return torch.sigmoid(x)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GCN(in_channels=2*window, out_channels=1).to(device)
    criterion = nn.BCELoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    train_ratio = 0.75 
    train_size = int(train_ratio * len(data_list))
    train_data = data_list[:train_size]
    test_data = data_list[train_size:]

    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=32)

    num_epochs = 20
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index).squeeze()
            loss = criterion(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.num_graphs
        
        print(f'Epoch {epoch+1}/{num_epochs}: Loss = {total_loss / len(train_loader.dataset):.4f}')

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.edge_index).squeeze()
            predicted_labels = (out > 0.5).long()
            correct += (predicted_labels == data.y).sum().item()
            total += data.y.size(0)
        
        accuracy = correct / total
        print(f'Test Accuracy: {accuracy:.4f}')

    del model
    torch.cuda.empty_cache()

test_2layer_GCN(data_df)

Epoch 1/20: Loss = 0.7533
Epoch 2/20: Loss = 0.4726
Epoch 3/20: Loss = 0.4101
Epoch 4/20: Loss = 0.3719
Epoch 5/20: Loss = 0.3410
Epoch 6/20: Loss = 0.3142
Epoch 7/20: Loss = 0.2892
Epoch 8/20: Loss = 0.2658
Epoch 9/20: Loss = 0.2531
Epoch 10/20: Loss = 0.2337
Epoch 11/20: Loss = 0.2175
Epoch 12/20: Loss = 0.2044
Epoch 13/20: Loss = 0.1935
Epoch 14/20: Loss = 0.1853
Epoch 15/20: Loss = 0.1780
Epoch 16/20: Loss = 0.1737
Epoch 17/20: Loss = 0.1675
Epoch 18/20: Loss = 0.1679
Epoch 19/20: Loss = 0.1660
Epoch 20/20: Loss = 0.1621
Test Accuracy: 0.9194


In [13]:
def test_2layer_GCN(df, window = 3):
    df_list = [df[i:i+window] for i in range(0,df.shape[0]-window+1,1)]

    df_dict = {}
    df_dict['x'] = [df_list[i].iloc[:,0:6] for i in range(len(df_list))]
    df_dict['y'] = [df_list[i].iloc[-1,-3:] for i in range(len(df_list))]

    
    def create_node_features(df_x):
        Prcp1 = df_x['Prcp1'].values
        Prcp2 = df_x['Prcp2'].values
        Prcp3 = df_x['Prcp3'].values
        SM1 = df_x['SM1'].values
        SM2 = df_x['SM2'].values
        SM3 = df_x['SM3'].values
        # Stack Prcp and SM for each node
        node_1 = np.hstack((Prcp1, SM1))
        node_2 = np.hstack((Prcp2, SM2))
        node_3 = np.hstack((Prcp3, SM3))
        node_features = np.vstack((node_1, node_2, node_3))
        
        return node_features

    data_list = []
    for i in range(len(df_dict['x'])):
        data_list.append(Data(
            x=torch.tensor(create_node_features(df_dict['x'][i]), dtype=torch.float), 
            y=torch.tensor(df_dict['y'][i].values, dtype=torch.float), 
            edge_index=torch.tensor([[0,2], [1,1]], dtype=torch.long)
            ))
        
    class GCN(nn.Module):
        def __init__(self, in_channels, out_channels):
            super(GCN, self).__init__()
            self.conv1 = GCNConv(in_channels, 16)  
            self.conv2 = GCNConv(16, out_channels)

        def forward(self, x, edge_index):
            x = self.conv1(x, edge_index)
            x = torch.relu(x)
            x = self.conv2(x, edge_index)
            return torch.sigmoid(x)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GCN(in_channels=2*window, out_channels=1).to(device)
    criterion = nn.BCELoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    train_ratio = 0.75 
    train_size = int(train_ratio * len(data_list))
    train_data = data_list[:train_size]
    test_data = data_list[train_size:]

    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=32)

    num_epochs = 50
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index).squeeze()
            loss = criterion(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.num_graphs
        
        print(f'Epoch {epoch+1}/{num_epochs}: Loss = {total_loss / len(train_loader.dataset):.4f}')

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.edge_index).squeeze()
            predicted_labels = (out > 0.5).long()
            correct += (predicted_labels == data.y).sum().item()
            total += data.y.size(0)
        
        accuracy = correct / total
        print(f'Test Accuracy: {accuracy:.4f}')

    del model
    torch.cuda.empty_cache()

test_2layer_GCN(data_df)

Epoch 1/50: Loss = 1.4361
Epoch 2/50: Loss = 0.7346
Epoch 3/50: Loss = 0.5307
Epoch 4/50: Loss = 0.4375
Epoch 5/50: Loss = 0.3869
Epoch 6/50: Loss = 0.3518
Epoch 7/50: Loss = 0.3225
Epoch 8/50: Loss = 0.2996
Epoch 9/50: Loss = 0.2810
Epoch 10/50: Loss = 0.2616
Epoch 11/50: Loss = 0.2450
Epoch 12/50: Loss = 0.2282
Epoch 13/50: Loss = 0.2192
Epoch 14/50: Loss = 0.2077
Epoch 15/50: Loss = 0.1942
Epoch 16/50: Loss = 0.1905
Epoch 17/50: Loss = 0.1875
Epoch 18/50: Loss = 0.1746
Epoch 19/50: Loss = 0.1823
Epoch 20/50: Loss = 0.1714
Epoch 21/50: Loss = 0.1645
Epoch 22/50: Loss = 0.1631
Epoch 23/50: Loss = 0.1640
Epoch 24/50: Loss = 0.1662
Epoch 25/50: Loss = 0.1581
Epoch 26/50: Loss = 0.1578
Epoch 27/50: Loss = 0.1556
Epoch 28/50: Loss = 0.1543
Epoch 29/50: Loss = 0.1613
Epoch 30/50: Loss = 0.1476
Epoch 31/50: Loss = 0.1490
Epoch 32/50: Loss = 0.1484
Epoch 33/50: Loss = 0.1538
Epoch 34/50: Loss = 0.1513
Epoch 35/50: Loss = 0.1508
Epoch 36/50: Loss = 0.1461
Epoch 37/50: Loss = 0.1461
Epoch 38/5

In [14]:
def test_2layer_GCN(df, window = 3):
    df_list = [df[i:i+window] for i in range(0,df.shape[0]-window+1,1)]

    df_dict = {}
    df_dict['x'] = [df_list[i].iloc[:,0:6] for i in range(len(df_list))]
    df_dict['y'] = [df_list[i].iloc[-1,-3:] for i in range(len(df_list))]

    
    def create_node_features(df_x):
        Prcp1 = df_x['Prcp1'].values
        Prcp2 = df_x['Prcp2'].values
        Prcp3 = df_x['Prcp3'].values
        SM1 = df_x['SM1'].values
        SM2 = df_x['SM2'].values
        SM3 = df_x['SM3'].values
        # Stack Prcp and SM for each node
        node_1 = np.hstack((Prcp1, SM1))
        node_2 = np.hstack((Prcp2, SM2))
        node_3 = np.hstack((Prcp3, SM3))
        node_features = np.vstack((node_1, node_2, node_3))
        
        return node_features

    data_list = []
    for i in range(len(df_dict['x'])):
        data_list.append(Data(
            x=torch.tensor(create_node_features(df_dict['x'][i]), dtype=torch.float), 
            y=torch.tensor(df_dict['y'][i].values, dtype=torch.float), 
            edge_index=torch.tensor([[0,2], [1,1]], dtype=torch.long)
            ))
        
    class GCN(nn.Module):
        def __init__(self, in_channels, out_channels):
            super(GCN, self).__init__()
            self.conv1 = GCNConv(in_channels, 16)  
            self.conv2 = GCNConv(16, out_channels)

        def forward(self, x, edge_index):
            x = self.conv1(x, edge_index)
            x = torch.relu(x)
            x = self.conv2(x, edge_index)
            return torch.sigmoid(x)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GCN(in_channels=2*window, out_channels=1).to(device)
    criterion = nn.BCELoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    train_ratio = 0.75 
    train_size = int(train_ratio * len(data_list))
    train_data = data_list[:train_size]
    test_data = data_list[train_size:]

    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=32)

    num_epochs = 100
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index).squeeze()
            loss = criterion(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.num_graphs
        
        print(f'Epoch {epoch+1}/{num_epochs}: Loss = {total_loss / len(train_loader.dataset):.4f}')

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.edge_index).squeeze()
            predicted_labels = (out > 0.5).long()
            correct += (predicted_labels == data.y).sum().item()
            total += data.y.size(0)
        
        accuracy = correct / total
        print(f'Test Accuracy: {accuracy:.4f}')

    del model
    torch.cuda.empty_cache()

test_2layer_GCN(data_df)

Epoch 1/100: Loss = 0.7920
Epoch 2/100: Loss = 0.4956
Epoch 3/100: Loss = 0.4264
Epoch 4/100: Loss = 0.3731
Epoch 5/100: Loss = 0.3471
Epoch 6/100: Loss = 0.3233
Epoch 7/100: Loss = 0.3046
Epoch 8/100: Loss = 0.2715
Epoch 9/100: Loss = 0.2518
Epoch 10/100: Loss = 0.2371
Epoch 11/100: Loss = 0.2205
Epoch 12/100: Loss = 0.2114
Epoch 13/100: Loss = 0.2132
Epoch 14/100: Loss = 0.2033
Epoch 15/100: Loss = 0.1834
Epoch 16/100: Loss = 0.1722
Epoch 17/100: Loss = 0.1686
Epoch 18/100: Loss = 0.1628
Epoch 19/100: Loss = 0.1643
Epoch 20/100: Loss = 0.1626
Epoch 21/100: Loss = 0.1550
Epoch 22/100: Loss = 0.1558
Epoch 23/100: Loss = 0.1594
Epoch 24/100: Loss = 0.1517
Epoch 25/100: Loss = 0.1469
Epoch 26/100: Loss = 0.1394
Epoch 27/100: Loss = 0.1387
Epoch 28/100: Loss = 0.1367
Epoch 29/100: Loss = 0.1359
Epoch 30/100: Loss = 0.1304
Epoch 31/100: Loss = 0.1301
Epoch 32/100: Loss = 0.1337
Epoch 33/100: Loss = 0.1325
Epoch 34/100: Loss = 0.1289
Epoch 35/100: Loss = 0.1240
Epoch 36/100: Loss = 0.1211
E